In [1]:
DATE = "2022-01-31"
TASK = "coronavirus-metapangenome-graph"
DIR = mkpath("$(homedir())/workspace/$(DATE)-$(TASK)")
cd(DIR)

In [1]:
pkgs = [
    "Graphs",
    "MetaGraphs",
    "FileIO",
    "HTTP",
    "JLD2",
    "DataStructures",
    "BioSequences",
    "DataFrames",
    "Random",
    "FASTX",
    "Revise",
    "StatsBase",
    "uCSV"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/.julia/dev/Mycelia/docs/Project.toml`
  No Changes to `~/.julia/dev/Mycelia/docs/Manifest.toml`


download refseq metadata file

In [2]:
refseq_summary_url = "https://ftp.ncbi.nih.gov/genomes/refseq/assembly_summary_refseq.txt"
# drop first line which is a comment
refseq_summary_table = DataFrames.DataFrame(uCSV.read(IOBuffer(HTTP.get(refseq_summary_url).body), comment = "#  ", header=1, delim='\t')...)

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCF_000001215.4,PRJNA164,SAMN02803731,,reference genome
2,GCF_000001405.39,PRJNA168,,,reference genome
3,GCF_000001635.27,PRJNA169,,,reference genome
4,GCF_000001735.4,PRJNA116,SAMN03081427,,reference genome
5,GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome
6,GCF_000001985.1,PRJNA32665,SAMN02953685,ABAR00000000.1,representative genome
7,GCF_000002035.6,PRJNA13922,SAMN06930106,,reference genome
8,GCF_000002075.1,PRJNA209509,SAMN02953658,AASC00000000.3,representative genome
9,GCF_000002235.5,PRJNA13728,SAMN00829422,AAGJ00000000.6,representative genome


In [3]:
show(refseq_summary_table[1:1, :], allcols=true)

1×23 DataFrame
 Row │ # assembly_accession  bioproject  biosample     wgs_master  refseq_category   taxid  species_taxid  organism_name            infraspecific_name  isolate  version_status  assembly_level  release_type  genome_rep  seq_rel_date  asm_name                submitter                          gbrs_paired_asm  paired_asm_comp  ftp_path                           excluded_from_refseq  relation_to_type_material  asm_not_live_date 
     │ String                String      String        String      String            Int64  Int64          String                   String              String   String          String          String        String      String        String                  String                             String           String           String                             String                String                     String            
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Get the tax_id of all entities in NCBI that are virus or below

In [4]:
# # all viruses
# https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&id=10239&lvl=3&lin=f&keep=1&srchmode=1&unlock
# taxid = "10239"
# # coronavirus
# https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?lvl=0&amp;id=11118
taxid = "11118"

"11118"

In [5]:
NEO4J_BIN_DIR = "/home/jupyter-cjprybol/software/neo4j-community-4.4.3/bin"
if !occursin(NEO4J_BIN_DIR, ENV["PATH"])
    ENV["PATH"] = "$(NEO4J_BIN_DIR):" * ENV["PATH"]
end 
DOMAIN = "ncbi-taxonomy.cjp.garden"
# DOMAIN="34.138.169.173"
USERNAME="neo4j"
PASSWORD=readline(joinpath(homedir(), ".config", "neo4j", "ncbi-taxonomy.password.txt"));
DATABASE = "neo4j"
# ADDRESS="neo4j://$(DOMAIN):7687"
ADDRESS="bolt://$(DOMAIN):7687"

"bolt://ncbi-taxonomy.cjp.garden:7687"

In [15]:
cmd = "MATCH (n)<-[*]-(n2) WHERE n.tax_id IS NOT NULL AND n.tax_id = \"$(taxid)\" RETURN DISTINCT n2.tax_id AS tax_id"
println(cmd)

MATCH (n)<-[*]-(n2) WHERE n.tax_id IS NOT NULL AND n.tax_id = "11118" RETURN DISTINCT n2.tax_id AS tax_id


In [16]:
coronaviridae_tax_ids = readlines(Mycelia.cypher(address = ADDRESS, username = USERNAME, password = PASSWORD, database = DATABASE, cmd=cmd))[2:end]
coronaviridae_tax_ids = strip.(coronaviridae_tax_ids, '"')
coronaviridae_tax_ids = vcat(coronaviridae_tax_ids, taxid)

1808-element Vector{AbstractString}:
 "693995"
 "2664420"
 "681862"
 "681861"
 "681883"
 "681864"
 "681878"
 "681870"
 "681849"
 "681871"
 "681880"
 "681852"
 "681876"
 ⋮
 "2078577"
 "2219863"
 "2249315"
 "2707191"
 "2724194"
 "2730119"
 "2050019"
 "2501930"
 "2788713"
 "2830724"
 "2836890"
 "11118"

In [18]:
refseq_summary_table[!, "taxid"]

251222-element Vector{Int64}:
    7227
    9606
   10090
    3702
    9785
  441960
    7955
    6500
    7668
    9615
   13616
    7070
    5855
       ⋮
     571
     573
 1134687
    9662
  999141
 2897682
 2905650
 2905649
 2905647
 2905648
     197
     197

In [ ]:
$ `a

$ cd pantools
$ git checkout pantools_v3.2
$ java -jar pantools/dist/pantools.jar # test if PanTools is executable

#Set an alias in your ~/.bashrc
$ echo "alias pantools='/mnt/YOUR_PATH/jdk1.8.0_161/bin/java -Xms50g -Xmx50g -jar /YOUR_PATH/pantools/dist/pantools.jar'" >> ~/.bashrc

In [ ]:
build a graph database of it's ref sequence, it's annotations, it's DNA, RNA, and AA nodes

nodes:
    type:
    object:
    key1 - value1
    key2 - value2
    
edges:
    use neo4j metagraph model

In [ ]:
need to be able to do this for genbank and fasta + GFF, and do it for a single genome and 3 genomes

In [ ]:
AAmer - 1, 3, 5, 7?
RNAmer - 1, 3, 5, 7, 9, 11, 13, 17, 23, 31, 41, 53
DNAmer - 1, 3, 5, 7, 9, 11, 13, 17, 23, 31, 41, 53